In [7]:
import numpy as np
import pandas as pd
from itertools import permutations, product
from analyzer import Analyzer
from tools import *

In [8]:
for dataset in ['dogs3', 'birds5']:
    df_s = {}
    for num in [2, 3, 4, 5, 6, 7, 8]:
        config_count = {}
        file = f'sd-rq-wp-{num}-{dataset}-2023'
        np.random.seed(0)
        analyer = Analyzer("./data/{}.json".format(file), size_of_dataset(dataset), retrieve_true_adj_matrix(dataset)(), radio=False, dataset=dataset, practice=True)

        for worker_answers in analyer.workers_answers:
            for answer in worker_answers.worker_answers:
                # how the quesiton is presented to the worker
                ori = answer.original_order

                true_config = str(tuple(map(retrieve_get_true_cluster_id(dataset), ori)))

                if true_config not in config_count:
                    config_count[true_config] = {}


                ori_to_ori_id = dict(zip(ori, range(len(ori))))
                # how the worker answers the question
                ans = answer.clusters
                obs_config = str(sorted((map(lambda clus: list(map(lambda x: chr(x + 97), list(sorted(map(lambda itm: ori_to_ori_id[itm], clus))))), ans))))[1:-1].replace("'", "")

                config_count[true_config][obs_config] = config_count[true_config].get(obs_config, 0) + 1

        df_s[num] = pd.DataFrame(config_count).fillna(0).astype(int).sort_index(axis=1)

    prob_mats = {}
    for num in df_s.keys():
        prob_mats[num] = df_s[num] / df_s[num].sum(axis=0)

    for num in df_s.keys():
        prob_mats[num].to_pickle(f"./pkls/emp_obs_prob_mat_{num}_{dataset}.pkl")
        df_s[num].to_pickle(f"./pkls/emp_obs_mat_{num}_{dataset}.pkl")

In [9]:
dataset = 'birds5'
emp_obs_prob_mats = {}
emp_obs_mats = {}
for num in [3]:
    emp_obs_prob_mats[num] = pd.read_pickle(f"./pkls/emp_obs_prob_mat_{num}_{dataset}.pkl")
    emp_obs_mats[num] = pd.read_pickle(f"./pkls/emp_obs_mat_{num}_{dataset}.pkl")

In [10]:
emp_obs_mats[3][["(1, 1, 1)", "(1, 1, 0)", "(1, 0, 1)", "(0, 1, 1)", "(1, 1, 2)", "(1, 2, 1)", "(2, 1, 1)"]]

,"(1, 1, 1)","(1, 1, 0)","(1, 0, 1)","(0, 1, 1)","(1, 1, 2)","(1, 2, 1)","(2, 1, 1)"
"[a, b], [c]",5,14,2,3,10,3,6
"[a], [b], [c]",8,20,12,16,19,8,7
"[a, b, c]",8,2,6,0,4,9,2
"[a, c], [b]",7,2,16,0,4,13,3
"[a], [b, c]",4,1,0,16,2,2,7


In [11]:
emp_obs_prob_mats[3][["(1, 1, 1)", "(1, 1, 0)", "(1, 0, 1)", "(0, 1, 1)", "(1, 1, 2)", "(1, 2, 1)", "(2, 1, 1)"]]

,"(1, 1, 1)","(1, 1, 0)","(1, 0, 1)","(0, 1, 1)","(1, 1, 2)","(1, 2, 1)","(2, 1, 1)"
"[a, b], [c]",0.15625,0.358974,0.055556,0.085714,0.256410,0.085714,0.24
"[a], [b], [c]",0.25000,0.512821,0.333333,0.457143,0.487179,0.228571,0.28
"[a, b, c]",0.25000,0.051282,0.166667,0.000000,0.102564,0.257143,0.08
"[a, c], [b]",0.21875,0.051282,0.444444,0.000000,0.102564,0.371429,0.12
"[a], [b, c]",0.12500,0.025641,0.000000,0.457143,0.051282,0.057143,0.28
